# Web Scraping - Assignment 1 (Part 2)

In [1]:
from __future__ import print_function
import urllib3
from bs4 import BeautifulSoup, SoupStrainer, Comment
import unicodedata

import warnings
warnings.filterwarnings('ignore')

In [2]:
#storing the url into a variable
url = "https://en.wikipedia.org/wiki/Google"
baseurl = "wikipedia"

# getting the response status of the url
http = urllib3.PoolManager();
response = http.request('GET',url)
response.status

200

In [3]:
soup = BeautifulSoup(response.data,'lxml')

In [4]:
# extracting links in the url and storing in an array 'links'
import re
links = soup.find_all('a', attrs={'href': re.compile("^http")})
for link in links:
    print(link.get('href'))

http://google.com
https://www.google.com/intl/en/about/company/
https://web.archive.org/web/20110628231125/http://www.informationweek.com/news/internet/google/210603678
http://www.informationweek.com/news/internet/google/210603678
https://www.google.com/about/jobs/locations/
http://investor.google.com/proxy.html
https://www.bloomberg.com/news/articles/2017-09-01/alphabet-wraps-up-reorganization-with-a-new-company-called-xxvi
https://www.independent.co.uk/news/business/news/worlds-most-valuable-brands-facebook-google-apple-amazon-a7556571.html
https://www.webcitation.org/61rJXKAeq?url=http://alan.blog-city.com/an_evening_with_googles_marissa_mayer.htm
http://alan.blog-city.com/an_evening_with_googles_marissa_mayer.htm
https://web.archive.org/web/20120401005940/http://www.google.com/about/company/history/
https://www.google.com/about/company/history/
http://ilpubs.stanford.edu:8090/422/
https://www.google.com/corporate/tech.html
https://web.archive.org/web/20020506051802/http://www-digli

In [5]:
# getting the length of the links array
print(len(links))

582


In [6]:
# extracting the external links
exlinks=[]
for link in links:
    if baseurl not in link.get('href'):
        exlinks.append(link)

In [7]:
# getting the length of the array
print(len(exlinks))

437


In [8]:
# function to check if the link is valid
def check_valid_link(href):
    try:
        response = http.request('GET',href)
        if(response.status == 200):
            return("Valid")
        return("Invalid")
    except:
        return("Invalid")

In [9]:
# testing the function for a single href link
href1 = exlinks[0].get('href')
print(href1)
print(check_valid_link(href1))

http://google.com
Valid


In [10]:
from urllib.parse import urlparse
import requests
import time

# function to get datetime for a url
def get_last_modified(url):
    result = urlparse(url)
    if check_valid_link(url) == "Valid":
        if True if [result.scheme, result.netloc, result.path] else False:
            header = requests.head(url).headers
            if 'Last-Modified' in header:
                return header['Last-Modified']
            return "Data is not available"
        return "Data is not available"
    return "Data is not available"

In [11]:
#writing the external links, description, if the lnk is valid or not, and the datetime into a CSV file

import csv
import re
filename = "WebScrapingLinks.csv"
f = open(filename,"w")

# writing headers into the file
headers = "link,description,link_validity,datetime\n"
f.write(headers)

for link in exlinks:
    try:
        if(link.has_attr('href')):
            href = link.get('href')
            valid = check_valid_link(href)
            desc = link.text
            desc = desc.replace(",","")    
#             desc = unicodedata.normalize('NFKD',desc).encode('ascii', 'ignore')
            datetime = get_last_modified(href).replace(",","")
            f.write(href.replace(",","") + "," + desc + "," + valid + "," + datetime + "\n")
    except:
        pass
f.close()